<a href="https://colab.research.google.com/github/ankit-rathi/Quantvesting_v2/blob/main/myStocks_Portfolio_Analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install pyxirr
!pip install ta
!pip install yfinance==0.2.59

  Installing build dependencies ... -

 \

 done


  Getting requirements to build wheel ... -

 done


...

 done


 done
  Created wheel for ta: filename=ta-0.11.0-py3-none-any.whl size=29482 sha256=65fd13867b01fa00e19af73faf08457aab2b5558d3402e15c292f22188fe40dd
  Stored in directory: /home/runner/.cache/pip/wheels/e3/3a/ee/4955a26c90a4b7deb6d725dc8ec7b8604a7aef44e43a2e8af7
Successfully built ta


  Attempting uninstall: protobuf
    Found existing installation: protobuf 6.32.1
    Uninstalling protobuf-6.32.1:
      Successfully uninstalled protobuf-6.32.1


  Attempting uninstall: yfinance
    Found existing installation: yfinance 0.2.65
    Uninstalling yfinance-0.2.65:
      Successfully uninstalled yfinance-0.2.65
   ━━━━━━━━━━━━━━━━━━━━╺━━━━━━━━━━━━━━━━━━━ 1/2 [yfinance]

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2/2 [yfinance]


In [2]:
# import libraries

import numpy as np
import pandas as pd
import datetime
import warnings
warnings.filterwarnings('ignore')
import yfinance as yfin
import ta
import matplotlib.pyplot as plt
import requests

In [3]:
# notebook start time
import time
import datetime
import pytz

start_time = time.time()

# function to print date timestamp
def print_date_time():
  tz_NY = pytz.timezone('Asia/Kolkata')
  datetime_NY = datetime.datetime.now(tz_NY)
  print("Run date time (IST):", datetime_NY.strftime("%Y-%m-%d %H:%M:%S"))

In [4]:
# function to get booked and reserved amount
def get_amt():
  # fetch the JSON data from the URL
  url = "https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myPortfolioAmts.json"
  response = requests.get(url)
  pf_amts = response.json()  # parse the JSON data

  # extract values from the JSON
  py_booked_amt_dm = pf_amts["py_booked_amt_dm"]
  py_booked_amt_sv = pf_amts["py_booked_amt_sv"]
  cy_booked_amt_dm = pf_amts["cy_booked_amt_dm"]
  cy_booked_amt_sv = pf_amts["cy_booked_amt_sv"]
  reserve_amt_dm = pf_amts["reserve_amt_dm"]
  reserve_amt_sv = pf_amts["reserve_amt_sv"]

  # perform calculations
  py_booked_amt = py_booked_amt_dm + py_booked_amt_sv
  cy_booked_amt = cy_booked_amt_dm + cy_booked_amt_sv
  reserve_amt = reserve_amt_dm + reserve_amt_sv
  total_booked_amt = py_booked_amt + cy_booked_amt

  return total_booked_amt, reserve_amt, cy_booked_amt, py_booked_amt

gen_output = 0

In [5]:
# function to format the amount
def format_amt(number):
    abs_number = abs(number)

    if abs_number >= 1_00_00_000:  # Crores
        formatted_amt = f"{number / 1_00_00_000:.2f} C"
    elif abs_number >= 1_00_000:  # Lakhs
        formatted_amt = f"{number / 1_00_000:.2f} L"
    elif abs_number >= 1_000:  # Thousands
        formatted_amt = f"{number / 1_000:.2f} K"
    else:
        formatted_amt = f"{number:.2f}"

    return formatted_amt

# function to set start and end date
def get_start_end_date():
  start_date = (datetime.date.today() + datetime.timedelta(days=-365)).strftime('%Y-%m-%d')
  end_date = (datetime.date.today() + datetime.timedelta(days=1)).strftime('%Y-%m-%d')
  return start_date, end_date

# function to fetch my portfolio csv
def get_mypfs_df():
  mypfs_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myPortfolioStocks.csv')
  return mypfs_df

# function to fetch my prospects csv
def get_mypps_df():
  mypps_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myProspectsScrips.csv')
  return mypps_df

# function to fetch screener data
def get_myscreen_df():
  myscreen_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myScreenerDB.csv')
  return myscreen_df

# function to fetch momentum data
def get_myinvmt_df():
  myinvmt_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myInvestments.csv')
  return myinvmt_df

# function to get the stock ids
def get_stock_ids(df_pf):
  stock_n100 = df_pf['Symbol'].unique()

  exclude = ['CADILAHC','MMTC', 'MASFIN']

  stock_ids = df_pf[~df_pf['Symbol'].isin(exclude) ]['Symbol'].unique()

  #mypf = mypf[mypf.Forecast.notnull()]
  #stock_ids = mypf['Symbol'].unique()

  stock_ids.sort()
  return stock_ids

# get features from screener data
def get_screener_features():
  myscreen_df = get_myscreen_df()
  cols = ['Symbol', 'EPS', 'MedPE', 'ROCE%', 'ROE%', 'CapType']
  return myscreen_df[cols]

# get relative strength
def get_relative_strength(stock_list):

    # Dictionary to store stock tickers and their corresponding percentage price change
    stock_changes = {}

    # Loop through each stock and fetch its price data
    for stock in stock_list:
        try:
            # Download the stock data for the given date range
            stock_data = yfin.Ticker(stock + '.NS').history(period='1mo', interval='1d')[map(str.title, ['open', 'close', 'low', 'high', 'volume'])]

            # Calculate the percentage change for the stock
            if len(stock_data) > 0:
                start_price = stock_data['Close'].iloc[0]
                end_price = stock_data['Close'].iloc[-1]
                percent_change = round(((end_price - start_price) / start_price) * 100, 2)
                stock_changes[stock] = percent_change
            else:
                stock_changes[stock] = np.nan  # If no data is available, set to NaN

        except Exception as e:
            print(f"Error fetching data for {stock}: {e}")
            stock_changes[stock] = np.nan

    # Create a DataFrame with stock tickers and their percentage changes
    df = pd.DataFrame(list(stock_changes.items()), columns=['Symbol', 'Percent_Change'])

    # Drop any stocks with missing data (NaN values)
    df = df.dropna()

    # Calculate the percentile rank based on percentage change
    df['RSP'] = round(df['Percent_Change'].rank(pct=True) * 100, 2)
    cols = ['Symbol', 'RSP']
    df = df[cols]
    # Sort by percentile rank (optional)
    df = df.sort_values(by='RSP', ascending=False).reset_index(drop=True)

    return df


In [6]:
# function to get stock technicals
def stock_prec_dev(stock_symbol):
    #stock_symbol = 'ULTRACEMCO.NS'
    short_window = 20
    mid_window = 50
    long_window = 200
    moving_avg = 'SMA'

    start = datetime.datetime(*map(int, start_date.split('-')))
    end = datetime.datetime(*map(int, end_date.split('-')))

    stock_df = yfin.Ticker(stock_symbol + '.NS').history(period='max', interval='1d')[map(str.title, ['open', 'close', 'low', 'high', 'volume'])]

    stock_df['Max'] = round(max(stock_df['Close']),2)
    stock_df = stock_df[(stock_df.index <= end_date) & (stock_df.index >= start_date)]
    stock_df['200_SMA'] = round(stock_df['Close'].rolling(window = 200, min_periods = 1).mean(),0)
    stock_df['Dev%_200'] = round((stock_df['Close'] - stock_df['200_SMA'])*100/stock_df['200_SMA'],2)
    stock_df.dropna(axis = 0, inplace = True) # remove any null rows

    stock_df['20_SMA'] = stock_df['Close'].rolling(window=20).mean()
    stock_df['50_SMA'] = stock_df['Close'].rolling(window=50).mean()
    stock_df['Symbol'] = stock_symbol

    stock_df['Close'] = round(stock_df['Close'],2)
    stock_df['Min'] = round(min(stock_df['Close']),2)
    stock_df['RSI_14'] = ta.momentum.RSIIndicator(close=stock_df['Close'], window=14).rsi()
    stock_df['RSI_14'] = round(stock_df['RSI_14'],0)
    stock_df['Prev_Close'] = stock_df['Close'].shift(1)
    stock_df.drop(['Open', 'Low', 'High', 'Volume'], axis=1, inplace=True)
    stock_df = stock_df.tail(1)

    max_SMA = max(stock_df['20_SMA'].item(), stock_df['50_SMA'].item(), stock_df['200_SMA'].item())
    min_SMA = min(stock_df['20_SMA'].item(), stock_df['50_SMA'].item(), stock_df['200_SMA'].item())
    ABS_Spread = max_SMA - min_SMA
    stock_df['Spread%'] = round((ABS_Spread / stock_df['200_SMA'].item()) * 100,2)

    return stock_df

# function to compute stock attributes
def get_common_features(stock_ids, df_mypf):

  df_prec_dev = pd.DataFrame()
  df_tmp = get_screener_features()
  df_rs = get_relative_strength(stock_ids)

  for stock_id in stock_ids:
      tmp = stock_prec_dev(stock_id)
      tmp = tmp.reset_index()
      df_prec_dev = pd.concat([df_prec_dev, tmp], ignore_index = True)
  df_prec_dev = pd.merge(df_prec_dev, df_mypf, on= 'Symbol')
  df_prec_dev = pd.merge(df_prec_dev, df_tmp, on= 'Symbol', how='left')
  df_prec_dev = pd.merge(df_prec_dev, df_rs, on= 'Symbol', how='left')
  #print(df_prec_dev.columns)
  df_prec_dev['Curr_PE'] = round(df_prec_dev['Close']/df_prec_dev['EPS'],1)
  df_prec_dev['Dev%_PE'] = round((df_prec_dev['Curr_PE'] - df_prec_dev['MedPE'])*100/df_prec_dev['MedPE'],2)
  df_prec_dev['Conviction'] = df_prec_dev['Conviction'] + '-' + df_prec_dev['CapType']
  return df_prec_dev

# function to arrange stock features
def arrange_features(df_stocks, common_cols, diff_cols):
  df_stocks_common = df_stocks[common_cols].drop_duplicates()
  df_stocks_diff = df_stocks[diff_cols]
  df_stocks_diff['Investment'] = df_stocks_diff['AvgCost'] * df_stocks_diff['Shares']
  df_stocks_diff = df_stocks_diff.groupby(['Symbol'])[['Shares', 'Investment']].aggregate(['sum']).reset_index()
  df_stocks_diff.columns = ['Symbol', 'Shares', 'Investment']
  df_stocks_diff['AvgCost'] = round(df_stocks_diff['Investment']/df_stocks_diff['Shares'],2)
  df_stocks = pd.merge(df_stocks_diff, df_stocks_common, on='Symbol')
  return df_stocks

# plot fact distribution across dimension
def plot_pie_chart(df, dimension, fact):
  # grouping the data by category and calculating the sum of fact for each type
  grouped_data = df.groupby(dimension)[fact].sum()

  # sorting the grouped data in descending order
  grouped_data = grouped_data.sort_values(ascending=False)

  # creating a pie chart
  grouped_data.plot.pie(autopct='%1.1f%%', startangle=90, figsize=(6, 6))

  # adding a title and displaying the plot
  plt.title(f'{dimension} {fact} Distribution')
  plt.ylabel('')  # To hide the y-label
  plt.show()

In [7]:
# function to get portfolio features

def get_portfolio_features(df_common_features):

  print_date_time()
  print('-------------------')

  #df_common_features["Target"] = df_common_features["Target"].fillna(df_common_features["Max"])
  df_common_features['NTT'] = np.where(df_common_features['Strategy']== 'NTT', df_common_features["Target"], df_common_features['Max'])
  df_common_features['LTT'] = np.where(df_common_features['Strategy']== 'BTT', df_common_features["Target"], df_common_features['Max'])
  df_common_features['BOL'] = df_common_features['Min']

  tmp_df = df_common_features[df_common_features['Symbol'].isin(mypf_df[mypf_df['InPortfolio'] != 'NA'].Symbol.values)]
  print('qualified stocks: '+str(len(tmp_df['Symbol'].unique())))
  tmp_df1 = tmp_df[tmp_df['LatestQtr'] == 1]
  print('with latest results: '+str(len(tmp_df1['Symbol'].unique())))
  tmp_df1 = tmp_df1[tmp_df1['StarStock'] == 1]
  print('still star stocks: '+str(len(tmp_df1['Symbol'].unique())))
  tmp_df['Investment'] = tmp_df['AvgCost'] * tmp_df['Shares']
  tmp_df['Current'] = round(tmp_df['Close'] * tmp_df['Shares'],0)
  tmp_df['Previous'] = tmp_df['Prev_Close'] * tmp_df['Shares']
  tmp_df['EstimatedST'] = tmp_df['NTT'] * tmp_df['Shares']
  tmp_df['EstimatedLT'] = tmp_df['LTT'] * tmp_df['Shares']
  tmp_df['Current P/L'] = round((tmp_df['Current'] - tmp_df['Investment']),0)
  tmp_df['Today P/L%'] = round((tmp_df['Current'] - tmp_df['Previous'])*100/tmp_df['Previous'],2)
  tmp_df['Current P/L%'] = round((tmp_df['Current'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['EstimatedST P/L%'] = round((tmp_df['EstimatedST'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['EstimatedLT P/L%'] = round((tmp_df['EstimatedLT'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['NTT%'] = round((tmp_df['NTT'] - tmp_df['Close'])*100/tmp_df['Close'],2)
  tmp_df['LTT%'] = round((tmp_df['LTT'] - tmp_df['Close'])*100/tmp_df['Close'],2)
  tmp_df['Gained%'] = round((tmp_df['Close'] - tmp_df['BOL'])*100/tmp_df['BOL'],2)
  investment = round(sum(tmp_df['AvgCost']*tmp_df['Shares']),0)
  current = round(sum(tmp_df['Close']*tmp_df['Shares']),0)
  tmp_df['InitAlloc%'] = round(tmp_df['Investment']*100/investment,2)
  tmp_df['CurrAlloc%'] = round(tmp_df['Current']*100/current,2)
  tmp_df['FTT'] = tmp_df['LTT']
  tmp_df.loc[tmp_df['Strategy'] == 'NTT', 'FTT'] = tmp_df['NTT']
  tmp_df['FTT%'] = tmp_df['LTT%']
  tmp_df.loc[tmp_df['Strategy'] == 'NTT', 'FTT%'] = tmp_df['NTT%']
  tmp_df['FTT Amt'] = round(tmp_df['FTT%'] * tmp_df['Current']/100,0)
  tmp_df['OTT%'] = round((tmp_df['FTT'] - tmp_df['AvgCost'])*100/tmp_df['AvgCost'],2)
  tmp_df['RRR Ind'] = round(tmp_df['Current P/L']/tmp_df['FTT Amt'],2)
  tmp_df['Risk Ind'] = round(tmp_df['Current P/L%']*tmp_df['CurrAlloc%'],0)

  return tmp_df

# function to print portfolio features
def print_portfolio_stats(df_portfolio_features, myinvmt_df):
  from pyxirr import xirr

  total_booked_amt, reserve_amt, cy_booked_amt, py_booked_amt = get_amt()

  dates = myinvmt_df['Date'].values
  dates = np.append(dates, datetime.date.today().strftime('%d-%b-%y'))
  investment = myinvmt_df['Investment'].values
  dates= pd.to_datetime(dates)

  current = round(sum(df_portfolio_features['Close']*df_portfolio_features['Shares']),0) + reserve_amt
  investment_xirr = np.append(investment, current)
  cagr = round(xirr(pd.DataFrame({"dates": dates, "amounts": investment_xirr}))*100,2)

  investment = -sum(investment)
  invested = round(sum(df_portfolio_features['AvgCost']*df_portfolio_features['Shares']),0) + reserve_amt
  previous = round(sum(df_portfolio_features['Prev_Close']*df_portfolio_features['Shares']),0) + reserve_amt
  cy_invested = investment + py_booked_amt

  today_pnl_amount = current-previous
  today_pnl_percentage = round((current-previous)*100/previous,2)

  curr_pnl_amount = current-invested
  curr_pnl_percentage = round((curr_pnl_amount)*100/(cy_invested),2)

  cy_pnl_amount = cy_booked_amt + curr_pnl_amount
  cy_pnl_percentage = round((cy_pnl_amount)*100/cy_invested,2)

  overall_pnl_amount = total_booked_amt + curr_pnl_amount
  overall_pnl_percentage = round((overall_pnl_amount)*100/investment,2)

  estimate_st = round(sum(df_portfolio_features['FTT']*df_portfolio_features['Shares']),0)  + reserve_amt
  est_st_pnl_amount = estimate_st-current
  est_st_pnl_percentage = round((est_st_pnl_amount)*100/current,2)

  estimate_lt = round(sum(df_portfolio_features['LTT']*df_portfolio_features['Shares']),0)  + reserve_amt
  est_lt_pnl_amount = estimate_lt-current
  est_lt_pnl_percentage = round((est_lt_pnl_amount)*100/current,2)

  total_profit = round(sum(df_portfolio_features[df_portfolio_features['Current P/L%'] > 0]['Current']) - sum(df_portfolio_features[df_portfolio_features['Current P/L%'] > 0]['Investment']),0)
  total_loss = round(sum(df_portfolio_features[df_portfolio_features['Current P/L%'] < 0]['Current']) - sum(df_portfolio_features[df_portfolio_features['Current P/L%'] < 0]['Investment']),0)

  cy_booked_percentage = round((cy_booked_amt)*100/current,2)
  py_booked_percentage = round((py_booked_amt)*100/investment,2)
  total_booked_percentage = round((total_booked_amt)*100/investment,2)

  print('-------------------')
  print('Initial Investment: ', format_amt(investment))
  print('CY Investment: ', format_amt(cy_invested))
  print('Reserve: ', format_amt(reserve_amt))
  print('Current: ',  format_amt(current))
  print('-------------------')
  print('Today PnL: '+ '{} ({}%)'.format(format_amt(today_pnl_amount), today_pnl_percentage))
  print('Current PnL: '+ '{} ({}%)'.format(format_amt(curr_pnl_amount), curr_pnl_percentage))
  print('CY Booked + Current PnL: '+ '{} ({}%)'.format(format_amt(cy_pnl_amount), cy_pnl_percentage))
  print('-------------------')
  print('Total profit: ', format_amt(total_profit))
  print('Total loss: ', format_amt(total_loss))
  print('-------------------')
  print('Total Booked + Current PnL: '+ '{} ({}%)'.format(format_amt(overall_pnl_amount), overall_pnl_percentage))
  print('Total Booked PnL: '+ '{} ({}%)'.format(format_amt(total_booked_amt), total_booked_percentage))
  print('Curr Year Booked PnL: '+ '{} ({}%)'.format(format_amt(cy_booked_amt), cy_booked_percentage))
  print('Prev Year Booked PnL: '+ '{} ({}%)'.format(format_amt(py_booked_amt), py_booked_percentage))
  print('===================')
  print('Est FTT: ',  format_amt(estimate_st))
  print('Est FTT PnL: '+ '{} ({}%)'.format(format_amt(est_st_pnl_amount), est_st_pnl_percentage))

  print('===================')
  print('Deployed: ', format_amt(investment))

  print('Current: ', format_amt(current))

  print('CAGR/XIRR %: '+'{}%'.format(cagr))

In [8]:
# get start and end date
start_date, end_date = get_start_end_date()

# get portfolio and prospects data
mypfs_df = get_mypfs_df()
mypps_df = get_mypps_df()
myinvmt_df = get_myinvmt_df()

# merge above datasets
mypf_df = pd.merge(mypfs_df, mypps_df, on="Symbol")

# seggregate dm and sv portfolio
dm_pf = mypf_df[mypf_df['InPortfolio'] == 'DM']
sv_pf = mypf_df[mypf_df['InPortfolio'] == 'SV']

# build portfolio stock dataframe
dm_stocks = get_stock_ids(dm_pf)
sv_stocks = get_stock_ids(sv_pf)
df_stocks = pd.concat([dm_pf,sv_pf], ignore_index = True)

# arrange common and diff stock features
common_cols = ['Symbol', 'Target', 'Criteria', 'Strategy', 'CumlRnk', 'LatestQtr', 'StarStock', 'Conviction', 'Category', 'InFolio', 'XIRR', 'MBQ']
diff_cols = ['Symbol', 'AvgCost', 'Shares']
df_stocks = arrange_features(df_stocks, common_cols, diff_cols)

# get common features
stock_ids = df_stocks['Symbol'].values
df_common_features = get_common_features(stock_ids, df_stocks)

df_common_features.reset_index(drop=True, inplace=True)
df_common_features.drop(['Date'], axis=1, inplace=True)
# get and print portfolio features
df_portfolio_features = get_portfolio_features(df_common_features)

print_portfolio_stats(df_portfolio_features, myinvmt_df)

df = df_portfolio_features
#plot_pie_chart(df, 'CapType', 'Current')
list_ox40n = list(df[df['MBQ'].str.contains('OX40N', na=False)]['Symbol'].values)

Run date time (IST): 2025-09-12 14:06:06
-------------------
qualified stocks: 87
with latest results: 27
still star stocks: 16
-------------------
Initial Investment:  1.22 C
CY Investment:  1.49 C
Reserve:  4.18 L
Current:  1.42 C
-------------------
Today PnL: 5.22 K (0.04%)
Current PnL: -17.54 L (-11.8%)
CY Booked + Current PnL: -6.51 L (-4.38%)
-------------------
Total profit:  2.10 L
Total loss:  -19.64 L
-------------------
Total Booked + Current PnL: 20.47 L (16.82%)
Total Booked PnL: 38.01 L (31.23%)
Curr Year Booked PnL: 11.03 L (7.79%)
Prev Year Booked PnL: 26.98 L (22.17%)
Est FTT:  2.25 C
Est FTT PnL: 83.65 L (59.05%)
Deployed:  1.22 C
Current:  1.42 C
CAGR/XIRR %: 9.49%


In [9]:
# top 5 near their targets
cols = ['Symbol', 'FTT', 'Dev%_PE', 'RSI_14','Conviction', 'Spread%', 'Current', 'Current P/L', 'FTT Amt', 'Today P/L%', 'Current P/L%', 'FTT%', 'OTT%', 'CumlRnk', 'RRR Ind', 'CurrAlloc%', 'Gained%', 'Criteria', 'Strategy', 'Category']
df_tmp = df_portfolio_features[~df_portfolio_features['Symbol'].isin(['ENRIN','BLUSPRING','DIGITIDE'])]
df_tmp = df_tmp.sort_values(by = 'FTT Amt', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
5,APOLLOHOSP,8285.00,-13.20,60.0,H-LC,11.19,173503.0,17326.0,8762.0,0.09,11.09,5.05,16.71,22.0,1.98,1.26,30.47,X40N,BTT,HEALTHCARE
62,SBIN,863.00,-11.34,61.0,M-LC,4.27,218241.0,16762.0,10454.0,-0.01,8.32,4.79,13.51,60.0,1.60,1.59,21.96,XY25,NTT,BANKS
77,TTKPRESTIG,770.00,103.42,57.0,M-SC,4.99,87685.0,-13092.0,13188.0,-0.56,-12.99,15.04,0.09,245.0,-0.99,0.64,15.20,OX40N,NTT,DURABLES
49,NATIONALUM,247.44,-42.00,73.0,H-MC,4.28,115487.0,14983.0,15660.0,1.63,14.91,13.56,30.49,79.0,0.96,0.84,52.14,MH,ATH,METALS
41,ITC,452.00,-37.08,56.0,H-LC,2.98,202764.0,2626.0,18269.0,-0.10,1.31,9.01,10.44,4.0,0.14,1.47,6.95,X40,NTT,FMCG


In [10]:
# top 5 to exit based on CumlRnk
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[(df_tmp['Current P/L%'] > -1) & (df_tmp['Current P/L%'] < 1)].sort_values(by = 'CumlRnk', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
35,ICICIPRULI,790.00,-21.11,41.0,H-MC,2.08,135690.0,-98.0,42851.0,0.33,-0.07,31.58,31.48,107.0,-0.00,0.99,12.04,X40,ATH,INSURANCE
30,HAVELLS,2069.16,-0.94,57.0,H-MC,1.68,245544.0,-2187.0,77248.0,-0.03,-0.88,31.46,30.30,92.0,-0.03,1.79,12.29,X40,ATH,ELECTRICAL


In [11]:
# top 5 to enter based on lowest RSI_14
df_tmp = df_portfolio_features[df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])].sort_values(by = 'RSI_14', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
83,VBL,671.64,-16.73,37.0,H-LC,6.05,299669.0,-16173.0,128828.0,-1.04,-5.12,42.99,35.67,5.0,-0.13,2.18,8.02,X40N,ATH,FMCG
43,JIOFIN,387.00,-1.56,43.0,H-LC,13.52,267589.0,224.0,65238.0,0.14,0.08,24.38,24.48,48.0,0.00,1.95,55.10,XY24,BTT,FINANCE
80,UNITDSPR,1644.00,-14.23,46.0,H-LC,8.06,224804.0,-9340.0,57954.0,-0.91,-3.99,25.78,20.77,86.0,-0.16,1.64,2.65,X40N,NTT,BREWERIES
31,HCLTECH,1943.91,1.08,49.0,H-LC,10.11,230476.0,-11448.0,74720.0,0.02,-4.73,32.42,26.15,8.0,-0.15,1.68,8.91,X40,ATH,IT
1,ABB,7934.00,-40.06,53.0,H-LC,11.96,250498.0,-11121.0,130334.0,0.01,-4.25,52.03,45.57,7.0,-0.09,1.82,6.59,AR,NTT,ELECTRICAL


In [12]:
# top 5 to enter based on Spread and CumlRnk
df_tmp = df_portfolio_features[df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[(df_tmp['CumlRnk'] < 100)].sort_values(by = 'Spread%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
47,LTIM,7230.20,-3.47,60.0,H-LC,1.18,239715.0,-6436.0,85650.0,0.67,-2.61,35.73,32.18,16.0,-0.08,1.74,32.49,X200,ATH,IT
41,ITC,452.00,-37.08,56.0,H-LC,2.98,202764.0,2626.0,18269.0,-0.10,1.31,9.01,10.44,4.0,0.14,1.47,6.95,X40,NTT,FMCG
50,NESTLEIND,1377.00,-7.46,62.0,H-LC,3.55,284872.0,19446.0,37347.0,-0.28,7.33,13.11,21.40,11.0,0.52,2.07,15.19,XY25,NTT,FMCG
20,CIPLA,1795.00,-19.09,56.0,H-LC,5.91,215638.0,11138.0,30276.0,0.85,5.45,14.04,20.25,10.0,0.37,1.57,14.66,X40N,BTT,PHARMA
83,VBL,671.64,-16.73,37.0,H-LC,6.05,299669.0,-16173.0,128828.0,-1.04,-5.12,42.99,35.67,5.0,-0.13,2.18,8.02,X40N,ATH,FMCG


In [13]:
# top 5 today
df_tmp = df_portfolio_features.sort_values(by = 'Today P/L%', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
8,ATULAUTO,844.00,4002.70,68.0,M-SC,7.13,127818.0,-19550.0,71361.0,11.76,-13.27,55.83,35.16,236.0,-0.27,0.93,30.55,XY24,NTT,AUTO
32,HINDZINC,730.22,22.76,67.0,M-LC,1.70,200404.0,-4672.0,117236.0,3.24,-2.28,58.50,54.89,52.0,-0.04,1.46,21.27,X5K,ATH,METALS
48,MASFIN,398.61,-14.63,63.0,H-SC,13.18,96960.0,-1020.0,22621.0,2.31,-1.04,23.33,22.05,152.0,-0.05,0.71,40.94,XR,ATH,FINANCE
58,ROUTE,2236.84,-46.24,49.0,H-SC,18.03,88188.0,-44878.0,137732.0,2.02,-33.73,156.18,69.78,118.0,-0.33,0.64,2.97,SR,ATH,IT
75,TITAGARH,1548.00,-2.10,65.0,H-SC,5.81,160650.0,-31365.0,110254.0,1.90,-16.33,68.63,41.08,158.0,-0.28,1.17,36.13,XY24,NTT,ENGINEERING


In [14]:
# bottom 5 today
df_tmp = df_portfolio_features.sort_values(by = 'Today P/L%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
4,ANGELONE,3033.0,2.55,32.0,H-SC,7.10,181712.0,-9294.0,66997.0,-5.16,-4.87,36.87,30.21,157.0,-0.14,1.32,15.00,X40N,NTT,FINANCE
53,RAJESHEXPO,518.0,1822.98,61.0,L-SC,5.56,52478.0,-84699.0,84794.0,-2.20,-61.74,161.58,0.07,267.0,-1.00,0.38,30.51,OX40N,NTT,JEWELLERY
67,SONACOMS,1006.0,-39.20,41.0,M-SC,11.99,75714.0,-25445.0,100336.0,-1.66,-25.15,132.52,74.03,202.0,-0.25,0.55,6.76,AR,BTT,AUTO
27,GREENPANEL,537.0,246.90,73.0,M-SC,5.36,154745.0,-28333.0,107316.0,-1.57,-15.48,69.35,43.14,230.0,-0.26,1.13,45.56,XY24,NTT,MISC
39,INFY,2275.0,-17.44,53.0,H-LC,9.75,321566.0,8678.0,163002.0,-1.49,2.77,50.69,54.87,3.0,0.05,2.34,9.61,X40,BTT,IT


In [15]:
# OX40N stocks
df_tmp = df_portfolio_features[(df_portfolio_features['Symbol'].isin(list_ox40n))].sort_values(by = 'Current P/L%', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
78,UJJIVANSFB,60.0,100.70,62.0,H-SC,12.58,126549.0,-15930.0,35446.0,-0.78,-11.18,28.01,13.70,163.0,-0.45,0.92,49.36,OX40N,NTT,BANKS
77,TTKPRESTIG,770.0,103.42,57.0,M-SC,4.99,87685.0,-13092.0,13188.0,-0.56,-12.99,15.04,0.09,245.0,-0.99,0.64,15.20,OX40N,NTT,DURABLES
44,KANSAINER,340.0,-67.42,61.0,H-SC,2.00,224154.0,-45513.0,81839.0,0.99,-16.88,36.51,13.47,138.0,-0.56,1.63,13.37,XY24,NTT,PAINTS
19,CERA,9475.0,-19.81,46.0,H-SC,2.14,146050.0,-29853.0,71871.0,0.13,-16.97,49.21,23.89,149.0,-0.42,1.06,26.13,OX40N,NTT,CERAMICS
72,TATAELXSI,9161.0,-20.01,52.0,H-MC,7.97,108129.0,-23706.0,65926.0,-1.07,-17.98,60.97,32.03,98.0,-0.36,0.79,21.11,OX40N,NTT,IT


In [16]:
# top 5 to accumulate
df_tmp = df_portfolio_features.sort_values(by = 'CumlRnk', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
74,TCS,4998.00,-26.44,54.0,H-LC,13.70,293675.0,-52281.0,176146.0,0.46,-15.11,59.98,35.80,1.0,-0.30,2.14,4.04,X40,BTT,IT
39,INFY,2275.00,-17.44,53.0,H-LC,9.75,321566.0,8678.0,163002.0,-1.49,2.77,50.69,54.87,3.0,0.05,2.34,9.61,X40,BTT,IT
41,ITC,452.00,-37.08,56.0,H-LC,2.98,202764.0,2626.0,18269.0,-0.10,1.31,9.01,10.44,4.0,0.14,1.47,6.95,X40,NTT,FMCG
83,VBL,671.64,-16.73,37.0,H-LC,6.05,299669.0,-16173.0,128828.0,-1.04,-5.12,42.99,35.67,5.0,-0.13,2.18,8.02,X40N,ATH,FMCG
1,ABB,7934.00,-40.06,53.0,H-LC,11.96,250498.0,-11121.0,130334.0,0.01,-4.25,52.03,45.57,7.0,-0.09,1.82,6.59,AR,NTT,ELECTRICAL


In [17]:
# top 5 to book for rotation
df_tmp = df_portfolio_features[(df_portfolio_features['Current P/L%'] > 20) ].sort_values(by = 'CumlRnk', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category


In [18]:
# top 5 to monitor
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[(df_tmp['Current P/L%'] > 0) ].sort_values(by = 'Dev%_PE', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
59,SAIL,228.00,44.35,64.0,M-MC,6.95,227666.0,2704.0,164944.0,0.30,1.20,72.45,74.53,192.0,0.02,1.66,33.84,XY24,BTT,STEEL
15,BLUESTARCO,2326.38,9.78,54.0,H-MC,3.97,189500.0,24830.0,43130.0,0.43,15.08,22.76,41.28,89.0,0.58,1.38,24.26,X40N,ATH,AC
84,VOLTAS,1918.49,0.18,56.0,H-MC,2.97,210135.0,18393.0,77645.0,0.40,9.59,36.95,50.08,99.0,0.24,1.53,17.04,XY25,ATH,AC
24,DABUR,735.00,-3.66,55.0,H-MC,6.16,209625.0,12597.0,77016.0,-1.21,6.39,36.74,45.49,102.0,0.16,1.52,20.27,XY24,BTT,FMCG
18,CAMS,5211.76,-5.38,50.0,H-SC,4.74,107654.0,5650.0,38271.0,-0.15,5.54,35.55,43.06,122.0,0.15,0.78,23.60,X40N,ATH,MISC


In [19]:
# top 5 to book for rotation from weak conviction
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[~df_tmp['Criteria'].isin(['XY25','XY24','X40', 'X40N'])].sort_values(by = 'RRR Ind', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
49,NATIONALUM,247.44,-42.00,73.0,H-MC,4.28,115487.0,14983.0,15660.0,1.63,14.91,13.56,30.49,79.0,0.96,0.84,52.14,MH,ATH,METALS
85,WHIRLPOOL,2270.00,-40.03,55.0,M-SC,1.41,101250.0,9752.0,69002.0,1.50,10.66,68.15,86.07,223.0,0.14,0.74,47.03,XR,NTT,DURABLES
36,INDIAMART,4850.92,-50.35,56.0,H-SC,12.28,138743.0,15407.0,118362.0,0.21,12.49,85.31,108.45,119.0,0.13,1.01,37.64,AR,ATH,MISC
86,WIPRO,420.00,-11.07,54.0,M-LC,5.80,156240.0,5295.0,104165.0,-0.75,3.51,66.67,72.51,53.0,0.05,1.14,9.74,XR,NTT,IT
12,BANKINDIA,190.00,-29.19,64.0,H-MC,5.50,181638.0,1830.0,110581.0,0.52,1.02,60.88,62.52,88.0,0.02,1.32,35.45,XR,NTT,BANKS


In [20]:
# top 5 for average up
df_tmp = df_portfolio_features[(df_portfolio_features['Dev%_200'] > 5)].sort_values(by = 'CurrAlloc%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
22,COFFEEDAY,80.00,-41.65,55.0,L-SC,44.50,86630.0,-26919.0,66965.0,-1.20,-23.71,77.30,35.27,268.0,-0.40,0.63,112.03,XR,NTT,HOTELS
3,ALKYLAMINE,4546.37,-5.85,42.0,H-SC,13.99,87604.0,-13359.0,107893.0,-0.62,-13.23,123.16,93.63,148.0,-0.12,0.64,30.48,SR,ATH,CHEMICALS
48,MASFIN,398.61,-14.63,63.0,H-SC,13.18,96960.0,-1020.0,22621.0,2.31,-1.04,23.33,22.05,152.0,-0.05,0.71,40.94,XR,ATH,FINANCE
49,NATIONALUM,247.44,-42.00,73.0,H-MC,4.28,115487.0,14983.0,15660.0,1.63,14.91,13.56,30.49,79.0,0.96,0.84,52.14,MH,ATH,METALS
82,VALIANTORG,838.00,-323.33,53.0,H-SC,7.78,124321.0,-39284.0,160598.0,0.18,-24.01,129.18,74.15,139.0,-0.24,0.90,59.36,XR,NTT,CHEMICALS


In [21]:
# top 5 RSP
df_tmp = df_portfolio_features.sort_values(by = 'RSP', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
8,ATULAUTO,844.00,4002.70,68.0,M-SC,7.13,127818.0,-19550.0,71361.0,11.76,-13.27,55.83,35.16,236.0,-0.27,0.93,30.55,XY24,NTT,AUTO
22,COFFEEDAY,80.00,-41.65,55.0,L-SC,44.50,86630.0,-26919.0,66965.0,-1.20,-23.71,77.30,35.27,268.0,-0.40,0.63,112.03,XR,NTT,HOTELS
49,NATIONALUM,247.44,-42.00,73.0,H-MC,4.28,115487.0,14983.0,15660.0,1.63,14.91,13.56,30.49,79.0,0.96,0.84,52.14,MH,ATH,METALS
27,GREENPANEL,537.00,246.90,73.0,M-SC,5.36,154745.0,-28333.0,107316.0,-1.57,-15.48,69.35,43.14,230.0,-0.26,1.13,45.56,XY24,NTT,MISC
75,TITAGARH,1548.00,-2.10,65.0,H-SC,5.81,160650.0,-31365.0,110254.0,1.90,-16.33,68.63,41.08,158.0,-0.28,1.17,36.13,XY24,NTT,ENGINEERING


In [22]:
# Top N allocation
df_tmp = df_portfolio_features.sort_values(by = 'CurrAlloc%', ascending=False)
top_n_values = [10, 25, 50]

sum_df = pd.DataFrame({
    'Top_N': top_n_values,
    'Sum_Alloc%': [df_tmp['CurrAlloc%'].head(n).sum() for n in top_n_values]
})

sum_df

,Top_N,Sum_Alloc%
0,10,19.72
1,25,43.46
2,50,73.87


In [23]:
# market-cap-wise allocation
df_tmp = df_portfolio_features[cols]
df_tmp.groupby(df_tmp['Conviction'].str[-2:])['CurrAlloc%'].sum().sort_values(ascending=False)

Conviction
SC    42.52
LC    34.49
MC    23.02
Name: CurrAlloc%, dtype: float64

In [24]:
# criteria-wise allocation
df_portfolio_features.groupby('Criteria')['CurrAlloc%'].sum().sort_values(ascending=False)

Criteria
XY24     29.02
X40      15.26
X40N     12.95
XR       10.04
XY25      9.76
AR        8.60
OX40N     8.20
X200      1.74
MH        1.72
X5K       1.46
SR        1.28
Name: CurrAlloc%, dtype: float64

In [25]:
# conviction-wise allocation
df_portfolio_features.groupby('Conviction')['CurrAlloc%'].sum().sort_values(ascending=False)

Conviction
H-SC    27.36
H-LC    26.24
H-MC    20.07
M-SC    13.61
M-LC     7.19
M-MC     2.64
L-SC     1.55
L-LC     1.06
L-MC     0.31
Name: CurrAlloc%, dtype: float64

In [26]:
# sector-wise stats
df_tmp = df_portfolio_features.groupby('Category')[['CurrAlloc%', 'Current', 'Current P/L', 'FTT Amt']].sum().sort_values(by=['Current', 'Current P/L'], ascending=False)
df_tmp['Current P/L%'] = round(df_tmp['Current P/L'] * 100 / df_tmp['Current'], 2)
df_tmp['FTT%'] = round(df_tmp['FTT Amt'] * 100 / df_tmp['Current'], 2)
cols = ['CurrAlloc%', 'Current P/L%', 'FTT%']
df_tmp[cols].sort_values(by=['CurrAlloc%'], ascending=False)

,CurrAlloc%,Current P/L%,FTT%
Category,,,
IT,13.37,-14.39,79.43
FMCG,11.81,-4.06,40.47
FINANCE,9.63,-16.46,62.08
BANKS,7.80,-11.95,63.48
MISC,6.09,-13.40,78.15
PAINTS,5.91,-13.54,38.95
ELECTRICAL,5.59,-7.25,66.36
AC,3.62,2.50,30.52
INSURANCE,3.40,-7.86,44.85


In [27]:
# money to be made criteria-wise
df_portfolio_features.groupby('Criteria')['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

,sum,count
Criteria,,
XY24,2857428.0,22
AR,1314477.0,9
XR,1233886.0,13
X40,869739.0,10
X40N,632111.0,10
OX40N,520554.0,10
XY25,409326.0,7
SR,245625.0,2
X5K,117236.0,1


In [28]:
# money to be made conviction-wise
df_portfolio_features.groupby('Conviction')['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

,sum,count
Conviction,,
H-SC,3353916.0,28
H-LC,1385660.0,15
M-SC,1377563.0,17
H-MC,1194258.0,15
M-LC,391509.0,5
M-MC,315066.0,2
L-SC,248194.0,3
L-MC,60194.0,1
L-LC,38204.0,1


In [29]:
# money to be made criteria and conviction-wise
df_portfolio_features.groupby(['Conviction', 'Criteria'])['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

sum  count
Conviction Criteria                  
H-SC       XY24      1039513.0      6
M-SC       XY24       790915.0      7
H-SC       AR         783067.0      5
           XR         679493.0      6
H-LC       X40        581717.0      5
H-MC       XY24       487042.0      4
H-LC       AR         368386.0      2
M-MC       XY24       315066.0      2
H-SC       X40N       297799.0      4
           SR         245625.0      2
           OX40N      245547.0      4
H-LC       X40N       225820.0      4
H-MC       X40        218472.0      4
           XY25       188085.0      2
L-SC       XR         163400.0      2
M-SC       AR         163024.0      2
M-LC       XY24       159654.0      2
M-SC       OX40N      124287.0      4
M-LC       X5K        117236.0      1
M-SC       XR         116053.0      2
           XY25       113734.0      1
H-MC       XR         110581.0      1
           X40N       108492.0      2
M-LC       XR         104165.0      1
H-LC       X200        85650.0      1
L-SC       OX40N       84794.0      1
M-SC       X40         69550.0      1
H-MC       OX40N       65926.0      1
H-LC       XY24        65238.0      1
H-SC       MH          62872.0      1
L-MC       XR          60194.0      1
H-LC       XY25        58849.0      2
L-LC       XY25        38204.0      1
H-MC       MH          15660.0      1
M-LC       XY25        10454.0      1

In [30]:
# notebook execution time

end_time = time.time()
execution_time = round(end_time - start_time, 0)
print(f"Notebook execution time: {execution_time} seconds")

Notebook execution time: 29.0 seconds
